In [2]:
from rdkit.Chem import Descriptors
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import rdDistGeom
from typing import List, Tuple
import numpy as np
import rdkit
import pandas as pd
from pymatgen.core import Structure, Lattice, Molecule

%load_ext autoreload
%autoreload 2

smiles = 'CC(C)C'

m = Chem.MolFromSmiles(smiles)
# necessary to add hydrogen for consistent conformer generation
m = Chem.AddHs(m)


Descriptors.Ipc(m)



312.0805106354346

In [28]:
import json
import gzip

from utils import download_url

#raw_url= "https://ml.materialsproject.org/projects/matbench_mp_is_metal.json.gz"
#raw_dir= "../data/matbench/mp_is_metal"
#download_url(raw_url, raw_dir)


json_filename = "../data/matbench/mp_is_metal/matbench_mp_is_metal.json.gz"


with gzip.open(json_filename, 'r') as fin:        # 4. gzip
    json_bytes = fin.read()                      # 3. bytes (i.e. UTF-8)

json_str = json_bytes.decode('utf-8')            # 2. string (i.e. JSON)
data = json.loads(json_str) 


from pymatgen.core.structure import Structure, Molecule

df = pd.DataFrame({"struct":[data_list[0] for data_list in data["data"]], "target":[data_list[1] for data_list in data["data"]]})

df.set_index("struct")

struct= Structure.from_dict(data["data"][0][0])

In [43]:
"json.gz" in json_filename

True

In [3]:
from pymatgen.core.lattice import Lattice

structure = Structure(
    Lattice.cubic(4.2),
    ["Na", "K", "K"],
    [[0, 0, 0], [0.5, 0.5, 0.5], [0,0,0.5]],
    site_properties={"magmom": [-2, 2,2]},
)

In [23]:
structure.formula

'K2 Na1'

In [3]:
from torch_geometric.data import Data


d = Data(x=[1,1], struct= structure)

d.struct

Structure Summary
Lattice
    abc : 4.2 4.2 4.2
 angles : 90.0 90.0 90.0
 volume : 74.08800000000001
      A : 4.2 0.0 0.0
      B : 0.0 4.2 0.0
      C : 0.0 0.0 4.2
    pbc : True True True
PeriodicSite: Na (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]
PeriodicSite: K (2.1000, 2.1000, 2.1000) [0.5000, 0.5000, 0.5000]
PeriodicSite: K (0.0000, 0.0000, 2.1000) [0.0000, 0.0000, 0.5000]

In [24]:
structure.distance_matrix

array([[0.        , 3.6373067 , 2.1       ],
       [3.6373067 , 0.        , 2.96984848],
       [2.1       , 2.96984848, 0.        ]])

In [26]:
a = np.array([0,0,1,1,2,2])
b = np.array([1,2,0,2,0,1])

structure.distance_matrix[a,b]

In [12]:
from utils import  Distance, from_structure_to_molecule, from_molecule_to_graph
import torch

atom_number_to_radius = torch.load("../important_data/atom_number_to_radius.pt")
    
distance = Distance(atom_number_to_radius=atom_number_to_radius)

from datasets_classes import BaceDataset

dataset = BaceDataset(root="../data/bace/")
graph = dataset[0]


#mol = from_structure_to_molecule(struct=structure,add_hydrogen=False)
#graph = from_molecule_to_graph(mol=mol, y=None, )

## given the pos, we need to see what dist is computed, maybe just give struct.frac_coords, need to check the format with real smiles molecule

(row, col), pos, pseudo = graph.edge_index, graph.pos, graph.edge_attr
dist = torch.norm(pos[col] - pos[row], p=2, dim=-1).view(-1, 1)